In [62]:
import json
from pathlib import Path
import pandas as pd

In [63]:
# ===== 경로 =====
PROJECT_DIR_PATH = Path.cwd()
DATA_DIR_PATH = PROJECT_DIR_PATH / "data"
OUTPUT_DIR_PATH = PROJECT_DIR_PATH / "output"
INTERNAL_DATA_DIR_PATH = DATA_DIR_PATH / "internal"
(OUTPUT_DIR_PATH / "jsonl").mkdir(parents=True, exist_ok=True)

xlsx_paths = [
    INTERNAL_DATA_DIR_PATH / "2. SRI시민패널조사" / "2024년 3분기 패널조사" / "(공개용) 2024년 3분기 패널조사 데이터(가중치).xlsx",
    INTERNAL_DATA_DIR_PATH / "2. SRI시민패널조사" / "2024년 4분기 패널조사" / "(공개용) 2024년 4분기 패널조사 데이터(가중치).xlsx",
    INTERNAL_DATA_DIR_PATH / "2. SRI시민패널조사" / "2025년 1분기 패널조사" / "(공개용) 2025년 1분기 패널조사 데이터(가중치).xlsx",
    INTERNAL_DATA_DIR_PATH / "2. SRI시민패널조사" / "2025년 2분기 패널조사" / "(공개용) 2025년 2분기 패널조사 데이터(가중치).xlsx",
]

codebook_paths = [
    OUTPUT_DIR_PATH / "codebooks" / "2024년 3분기 패널조사 데이터_코드북.json",
    OUTPUT_DIR_PATH / "codebooks" / "2024년 4분기 패널조사 데이터_코드북.json",  # ← 수정됨
    OUTPUT_DIR_PATH / "codebooks" / "2025년 1분기 패널조사_코드북.json",
    OUTPUT_DIR_PATH / "codebooks" / "2025년 2분기 패널조사_코드북.json",
]

In [64]:
# ==== 코드북 로드 ====
variables_list = []

for idx, codebook_path in enumerate(codebook_paths):
    with open(codebook_path, 'r', encoding='utf-8') as f:
        codebook_path = json.load(f)
    variables_list.append(codebook_path.get("variables", {}))

from pprint import pprint
pprint(variables_list[0])

{'DM1': {'choices': [{'code': 1, 'label': '남자'}, {'code': 2, 'label': '여자'}],
         'label': '성별',
         'name': 'DM1',
         'section': 'Ⅴ. 응답자 특성',
         'type': 'single',
         'unit': None},
 'DM10': {'choices': [{'code': 1, 'label': '일을 하였음'},
                      {'code': 2, 'label': '휴가 및 일시 휴직'},
                      {'code': 3, 'label': '일을 하지 않았음'}],
          'label': '일자리 경험_귀하는 지난 일주일 동안 수입을 목적으로 일을 한적 있습니까?',
          'name': 'DM10',
          'section': 'Ⅴ. 응답자 특성',
          'type': 'single',
          'unit': None},
 'DM11': {'choices': [{'code': 1, 'label': '상용직 임금근로자'},
                      {'code': 2, 'label': '임시직 임금근로자'},
                      {'code': 3, 'label': '일용직 임금근로자'},
                      {'code': 4, 'label': '특수고용 종사자'},
                      {'code': 5, 'label': '고용원이 있는 자영업자'},
                      {'code': 6, 'label': '고용원이 없는 자영업자'},
                      {'code': 7, 'label': '무급가족종사자'}],
          'label': '고용형태',
          'na

In [65]:
# ===== 공용 함수 =====
def choice_map(var_meta: dict):
    ch = var_meta.get("choices") if var_meta else None
    if not ch: 
        return None
    m = {c.get("code"): c.get("label") for c in ch}
    m |= {str(k): v for k, v in m.items()}  # 문자열 키도 허용
    return m

In [66]:
# ===== 행 -> JSON 변환 =====
def row_to_doc(row: pd.Series, variables: dict, quarter: str, source_file: str) -> dict:
    doc = {"quarter": quarter, "source_file": source_file}
    rid = row.get("ID")
    doc["respondent_id"] = None if pd.isna(rid) else str(rid)
    doc["_id"] = f"{quarter}-{doc['respondent_id']}"

    for col, val in row.items():
        if pd.isna(val):
            continue
        meta = variables.get(col)
        if meta and meta.get("type") == "single" and meta.get("choices"):
            cmap = choice_map(meta) or {}
            lbl = cmap.get(val, cmap.get(str(val)))
            doc[f"{col}_code"] = val
            if lbl is not None:
                doc[f"{col}_label"] = lbl
        else:
            if isinstance(val, float):
                doc[col] = float(val)
            elif isinstance(val, int):
                doc[col] = int(val)
            else:
                doc[col] = str(val)
    return doc

In [67]:
# ===== 실행: 분기별 JSONL + ALL JSONL =====
all_out_path = OUTPUT_DIR_PATH / "jsonl" / "ALL_quarters_panel.jsonl"
with open(all_out_path, "w", encoding="utf-8") as all_out:
    for xlsx_path, cb_path in zip(xlsx_paths, codebook_paths):
        # 코드북 로드
        with open(cb_path, "r", encoding="utf-8") as f:
            codebook = json.load(f)
        variables = codebook.get("variables", {})

        # 분기 이름 및 출력 경로
        quarter = xlsx_path.stem  # 예: "(공개용) 2024년 3분기 패널조사 데이터(가중치)"
        quarter_out = OUTPUT_DIR_PATH / "jsonl" / f"{quarter}.jsonl"

        # 변환
        df = pd.read_excel(xlsx_path)
        with open(quarter_out, "w", encoding="utf-8") as qout:
            for _, row in df.iterrows():
                doc = row_to_doc(row, variables, quarter, xlsx_path.name)
                line = json.dumps(doc, ensure_ascii=False)
                qout.write(line + "\n")
                all_out.write(line + "\n")

        print(f"[OK] {quarter_out}")

print(f"[OK] {all_out_path}")

[OK] d:\workspace\Bus_suwon\output\jsonl\(공개용) 2024년 3분기 패널조사 데이터(가중치).jsonl
[OK] d:\workspace\Bus_suwon\output\jsonl\(공개용) 2024년 4분기 패널조사 데이터(가중치).jsonl
[OK] d:\workspace\Bus_suwon\output\jsonl\(공개용) 2025년 1분기 패널조사 데이터(가중치).jsonl
[OK] d:\workspace\Bus_suwon\output\jsonl\(공개용) 2025년 2분기 패널조사 데이터(가중치).jsonl
[OK] d:\workspace\Bus_suwon\output\jsonl\ALL_quarters_panel.jsonl
